In [25]:
import simpy
import numpy as np

# Simulation parameters
SIM_DURATION = 30 * 12 * 60  # 30 days, 12 hours per day (8 AM to 8 PM)
CAPACITY = 200

# Arrival rates based on the day of the week and time slots
ARRIVAL_RATES = {
    'Sunday': [(8, 12, 75), (12, 17, 250), (17, 20, 250)],
    'Monday': [(8, 12, 75), (12, 17, 250), (17, 20, 250)],
    'Tuesday': [(8, 12, 75), (12, 17, 250), (17, 20, 250)],
    'Wednesday': [(8, 12, 75), (12, 17, 250), (17, 20, 250)],
    'Thursday': [(8, 12, 75), (12, 17, 250), (17, 20, 250)],
    'Friday': [(8, 12, 100), (12, 17, 350), (17, 20, 400)],
    'Saturday': [(8, 12, 100), (12, 17, 350), (17, 20, 400)]
}

# Initialize data for tracking arrivals and utilization
arrival_pattern = np.zeros((30, 12))  # 30 days, 12 hours per day (8 AM to 8 PM)
total_revenue = 0
lost_customers_due_to_price = 0
lost_customers_due_to_wtv = 0
lost_customers_due_to_capacity = 0
total_arrivals = 0  # Track total number of arrivals

# Utilization data storage
utilization_data_per_day = {day: [[] for _ in range(12)] for day in ARRIVAL_RATES.keys()}

def get_arrival_rate(day, hour):
    """Return the arrival rate based on the day and hour."""
    for start, end, rate in ARRIVAL_RATES[day]:
        if start <= hour < end:
            return rate
    return 0

def generate_wtv():
    """Generate a random WTV (Willingness to Visit) for each customer."""
    return np.random.uniform(0, 1)  # WTV is a probability between 0 and 1

def generate_willingness_to_pay():
    """Generate a random willingness to pay for each customer."""
    return np.random.uniform(15, 100)  # Assume willingness to pay ranges from ₹15 to ₹100

def calculate_charge(parking_duration):
    """Calculate the charge based on parking duration with ceiling logic."""
    if parking_duration < 30:
        return 15
    else:
        hours = np.ceil(parking_duration / 60)  # Round up to the nearest hour
        return 25 * hours  # Charge ₹25 per hour

def vehicle(env, parking_lot, day_index, hour_index, inter_arrival_time):
    """Simulates vehicle parking behavior, tracks arrivals, inter-arrival times, and calculates revenue."""
    global total_revenue, lost_customers_due_to_price, lost_customers_due_to_wtv, lost_customers_due_to_capacity, total_arrivals
    
    # Generate WTV for the customer
    wtv = generate_wtv()
    if wtv < 0.2:  # Assume customers with WTV < 0.2 are lost due to WTV
        lost_customers_due_to_wtv += 1
        return  # Customer leaves without parking
    
    # Generate parking duration (in minutes)
    parking_duration = np.random.exponential(60)  # Exponential distribution with mean 60 minutes
    
    # Calculate the charge based on parking duration
    charge = calculate_charge(parking_duration)
    
    # Generate customer's willingness to pay
    willingness_to_pay = generate_willingness_to_pay()
    if charge > willingness_to_pay:  # Customer is lost if the charge is higher than their willingness to pay
        lost_customers_due_to_price += 1
        return  # Customer leaves without parking

    # Update total revenue if the customer parks
    total_revenue += charge
    
    total_arrivals += 1  # Increment total arrivals count
    
    with parking_lot.request() as request:
        result = yield request | env.timeout(0)  # Request for parking space or timeout immediately if full
        
        # Log parking lot utilization
        current_time = env.now
        num_vehicles = len(parking_lot.users)
        day_name = list(ARRIVAL_RATES.keys())[day_index % 7]
        utilization_data_per_day[day_name][hour_index].append(num_vehicles)
        
        if request in result:
            # Vehicle successfully parks
            arrival_pattern[day_index, hour_index] += 1
            yield env.timeout(parking_duration)
        else:
            # No parking space available, track lost customer
            lost_customers_due_to_capacity += 1

def arrival_process(env, parking_lot, day, day_index):
    """Simulates the arrival process based on the day and time."""
    while env.now < (day_index + 1) * 12 * 60:  # Simulate only 12 hours per day
        current_hour = (env.now // 60) % 12 + 8  # Convert minutes to simulation hour (8 AM - 8 PM)
        rate = get_arrival_rate(day, current_hour)
        
        # Determine the hour index (0 to 11 for 8 AM to 8 PM)
        hour_index = int(current_hour - 8)  # Adjust index to match 0-11 range
        
        if 0 <= hour_index < 12:  # Ensure hour_index is within valid bounds
            if rate > 0:
                inter_arrival_time = np.random.exponential(60 / rate)  # Convert hourly rate to inter-arrival time
                yield env.timeout(inter_arrival_time)
                env.process(vehicle(env, parking_lot, day_index, hour_index, inter_arrival_time))
            else:
                yield env.timeout(1)  # Wait for 1 minute if no arrival

def run_simulation():
    """Runs the simulation for 30 days."""
    global total_revenue, lost_customers_due_to_price, lost_customers_due_to_wtv, lost_customers_due_to_capacity, total_arrivals
    env = simpy.Environment()
    parking_lot = simpy.Resource(env, capacity=CAPACITY)

    for day_index in range(30):
        day_name = list(ARRIVAL_RATES.keys())[day_index % 7]  # Cycle through days of the week
        env.process(arrival_process(env, parking_lot, day_name, day_index))

        # Simulate 12 hours of the current day
        env.run(until=(day_index + 1) * 12 * 60)

# Run the simulation
run_simulation()

# Print lost customer counts
print(f"Total Lost Customers Due to No Parking Space: {lost_customers_due_to_capacity}")
print(f"Total Lost Customers Due to Price: {lost_customers_due_to_price}")
print(f"Total Lost Customers Due to Low WTV: {lost_customers_due_to_wtv}")
print(f"Total Revenue: {total_revenue}")

# Calculate and print lost customers due to no parking space as a percentage
if total_arrivals > 0:
    lost_due_to_capacity_percentage = (lost_customers_due_to_capacity / total_arrivals) * 100
else:
    lost_due_to_capacity_percentage = 0

print(f"Percentage of Lost Customers Due to No Parking Space: {lost_due_to_capacity_percentage:.2f}%")

# Calculate and print average parking space utilization for different time slots and different days
for day, hours in utilization_data_per_day.items():
    print(f"\nAverage Parking Space Utilization for {day}:")
    for hour_index, utilizations in enumerate(hours):
        if utilizations:
            average_occupied_spaces = np.mean(utilizations)
            average_utilization = (average_occupied_spaces / CAPACITY) * 100
            print(f"  Hour {8 + hour_index}:00 - {8 + hour_index + 1}:00: {average_utilization:.2f}%")
        else:
            print(f"  Hour {8 + hour_index}:00 - {8 + hour_index + 1}:00: No data available")


Final Lost Customers Due to Full Capacity: 0
Final Percentage of Lost Customers Due to Full Capacity: 0.00%
Total Parked Customers: 14058
Total Revenue: ₹496350.00
